In [2]:
#!/usr/local/bin/python3
"""Blackboard Data API
Usage:
    bbdata.py [options]
    bbdata.py (-h | --help)
    bbdata.py --version

Commands:
    -s <store>, --store <store>  Set the persistence layer. [default: mongo]

Options:
General Options:
    -h, --help      Show this screen.
    -v, --version   Show version.
    -D, --debug     Enable debug mode: verbose.
    --pprint        Pretty print the results.
"""

'Blackboard Data API\nUsage:\n    bbdata.py [options]\n    bbdata.py (-h | --help)\n    bbdata.py --version\n\nCommands:\n    -s <store>, --store <store>  Set the persistence layer. [default: mongo]\n\nOptions:\nGeneral Options:\n    -h, --help      Show this screen.\n    -v, --version   Show version.\n    -D, --debug     Enable debug mode: verbose.\n    --pprint        Pretty print the results.\n'

In [3]:
import json
import sys
import re

from docopt import docopt
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
from pymongo import MongoClient, ReturnDocument, ASCENDING
from pprint import pprint
from settings import config as config, DEFAULT_MODE

In [4]:
def to_dict(result):
    return [dict(zip(row.keys(), row)) for row in result]


In [5]:
def load_query(query_name):
    query = None
    variables = None

    with open(f"./queries/{query_name}/query.sql") as qf:
        query = qf.read()

    with open(f"./queries/{query_name}/variables.json") as vf:
        variables = json.load(vf)['variables']

    for k, v in variables.items():
        query = query.replace('{' + k + '}', build_type(v))

    return to_dict(sessions['bbdata'].execute(query))


In [8]:
def build_type(value):
    if type(value) == dict:
        return json.dumps(value)
    if type(value) == list:
        return ','.join([build_type(i) for i in value])
    if type(value) == int:
        return value
    if type(value) == str:
        return f"'{value}'"


In [7]:
def main():
    load_user = load_query('get-user')

    print(load_user)

    for db, session in sessions.items():
        session.close()
        engines[db].dispose()

    print('Job Complete!!')

In [10]:
if __name__ == '__main__':

    args = docopt(__doc__, version='Blackboard Data API 0.0.1')

    store = args['--store']
    debug = args['--debug']

    client = None

    if store == 'mongo':
        # connect to mongo
        client = MongoClient(config['mongo'][DEFAULT_MODE]['url'])
        mongo_db = client[config['mongo'][DEFAULT_MODE]['database']]

    conn_str = f"snowflake://{config['bbdata']['user']}:{config['bbdata']['pass']}@{config['bbdata']['account']}/{config['bbdata']['database']}/{config['bbdata']['schema']}"
    engines = {
        'bbdata': create_engine(conn_str, echo=debug)
    }

    sessions = {
        'bbdata': sessionmaker(bind=engines['bbdata'])(),
    }

    output_file = './exports/GetUser.csv'
    log_file = './exports/ErrorLog.csv'
    main()


DocoptExit: Usage:
    bbdata.py [options]
    bbdata.py (-h | --help)
    bbdata.py --version